In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,369 kB]
Hit:13 http:

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://moviepickfiles.s3.us-east-2.amazonaws.com/combined_df.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("combined_df.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+---+---------+------------------+-----+------------+--------------------+--------------------+-------+------------+---------+----------+
|_c0|     Mood|    original_title|   id|release_year|               genre|            overview|runtime|vote_average|   budget|   revenue|
+---+---------+------------------+-----+------------+--------------------+--------------------+-------+------------+---------+----------+
|  0|    Happy|Back to the Future|  105|        1985|Adventure,Comedy,...|Eighties teenager...|  116.0|         8.0| 19000000| 381109762|
|  1|    Happy|      Forrest Gump|   13|        1994|Comedy,Drama,Romance|A man with a low ...|  142.0|         8.2| 55000000| 677945399|
|  2|      Sad|      Forrest Gump|   13|        1994|Comedy,Drama,Romance|A man with a low ...|  142.0|         8.2| 55000000| 677945399|
|  3|    Happy|      Forrest Gump|   13|        1994|Comedy,Drama,Romance|A man with a low ...|  142.0|         8.2| 55000000| 677945399|
|  4| Inspired|   The Dark Knight|

In [21]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [22]:
from pyspark import SparkFiles
import io

#data = io.BytesIO(uploaded['combined_df.csv'])
#data
#spark.sparkContext.addFile(io.BytesIO(uploaded['combined_df.csv']))
df = spark.read.csv('UWA DATA BOOTCAMP/Movie_pick/Output/combined_df.csv', sep=",", header=True)
df.show()

AnalysisException: ignored

In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['overview']))

data_df = data_df.drop('_c0',
 'original_title',
 'id',
 'release_year',
 'genre',
 'runtime',
 'vote_average',
 'budget',
 'revenue')

data_df = data_df.withColumnRenamed("Mood","mood")

data_df.show()


+---------+--------------------+------+
|     mood|            overview|length|
+---------+--------------------+------+
|    Happy|Eighties teenager...|   327|
|    Happy|A man with a low ...|   406|
|      Sad|A man with a low ...|   406|
|    Happy|A man with a low ...|   406|
| Inspired|Batman raises the...|   396|
| Inspired|Batman raises the...|   396|
|    Angry|Batman raises the...|   396|
|    Happy|An epic love stor...|   301|
| Romantic|An epic love stor...|   301|
|      Sad|An epic love stor...|   301|
|Emotional|Princess Leia is ...|   312|
|    Happy|"Bolt is the star...|   263|
|      Sad|Into the world of...|   153|
|    Angry|In a dystopian fu...|   117|
|    Angry|In a dystopian fu...|   117|
|    Happy|Nemo, an adventur...|   333|
| Inspired|Nemo, an adventur...|   333|
| Thrilled|When soldier Robi...|   266|
| Romantic|When the kingdom'...|   512|
|    Happy|When the kingdom'...|   512|
+---------+--------------------+------+
only showing top 20 rows



### Feature Transformations


In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='mood',outputCol='label')
tokenizer = Tokenizer(inputCol="overview", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

X = idf
y = pos_neg_to_num

In [ ]:
pos_neg_to_num

StringIndexer_20475dc49f44

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [ ]:
# Show label and resulting features
mood_index = cleaned.select(['mood','label', 'features'])

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(40)

+----------------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|            mood|            overview|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+----------------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|           Angry|Four young outsid...|   265|  5.0|[four, young, out...|[four, young, out...|(262144,[34343,36...|(262144,[34343,36...|(262145,[34343,36...|[-2185.5518156137...|[1.0,6.9859520425...|       0.0|
|           Angry|Fourteen hundred ...|   906|  5.0|[fourteen, hundre...|[fourteen, hundre...|(262144,[956,1619...|(262144,[956,1619...|(262145,[956,1619...

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.138730


In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://moviepickfiles.s3.us-east-2.amazonaws.com/movies_cleaned3.csv"
spark.sparkContext.addFile(url)
full_df = spark.read.csv(SparkFiles.get("movies_cleaned3.csv"), sep=",", header=True)

# Show DataFrame
full_df.show()

+------+--------------------+------------+--------------------+--------------------+-------+------------+--------------------+-----------------+--------------------+--------------------+--------------------+---------+----------+----------+--------------------+--------------------+
|    id|      original_title|release_year|               genre|            overview|runtime|vote_average|          keywording|original_language|      prod_companies|      prod_countries|        release_date|   budget|   revenue|popularity|    languages_spoken|             casting|
+------+--------------------+------------+--------------------+--------------------+-------+------------+--------------------+-----------------+--------------------+--------------------+--------------------+---------+----------+----------+--------------------+--------------------+
| 19995|              Avatar|        2009|Action,Adventure,...|In the 22nd centu...|    162|         7.2|culture clash,fut...|               en|Ingenious 

In [ ]:
from pyspark.sql.functions import lit

full_df = full_df.withColumn('length', length(full_df['overview']))
full_df = full_df.withColumn('mood',lit(0))

big_df = full_df.select('mood','overview','length')
big_df.show()

+----+--------------------+------+
|mood|            overview|length|
+----+--------------------+------+
|   0|In the 22nd centu...|   175|
|   0|Captain Barbossa,...|   176|
|   0|A cryptic message...|   239|
|   0|Following the dea...|   428|
|   0|John Carter is a ...|   341|
|   0|The seemingly inv...|   281|
|   0|When the kingdom'...|   506|
|   0|When Tony Stark t...|   393|
|   0|As Harry begins h...|   173|
|   0|Fearing the actio...|   370|
|   0|Superman returns ...|   307|
|   0|Quantum of Solace...|   375|
|   0|Captain Jack Spar...|   129|
|   0|The Texas Rangers...|   386|
|   0|A young boy learn...|   293|
|   0|One year after th...|   437|
|   0|When an unexpecte...|   297|
|   0|Captain Jack Spar...|   407|
|   0|Agents J (Will Sm...|   568|
|   0|Immediately after...|   412|
+----+--------------------+------+
only showing top 20 rows



In [ ]:
from pyspark.ml import Pipeline
data_prep_pipeline1 = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

cleaner1 = data_prep_pipeline1.fit(big_df)
cleaned1 = cleaner1.transform(big_df)

In [ ]:
test_results = predictor.transform(cleaned1)
test_results.show(100)

+----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|mood|            overview|length|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   0|In the 22nd centu...|   175|[in, the, 22nd, c...|[22nd, century,, ...|(262144,[19684,53...|(262144,[19684,53...|(262145,[19684,53...|[-1642.8725742871...|[1.0,1.8006093780...|       0.0|
|   0|Captain Barbossa,...|   176|[captain, barboss...|[captain, barboss...|(262144,[29562,41...|(262144,[29562,41...|(262145,[29562,41...|[-1740.7521795698...|[1.0,5.0305265759...|       0.0|
|   0|A cryptic message...|   239|[

In [ ]:
test_results = predictor.transform(cleaned)
test_results.show(5)

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    mood|            overview|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   Happy|Eighties teenager...|   327|  0.0|[eighties, teenag...|[eighties, teenag...|(262144,[30440,31...|(262144,[30440,31...|(262145,[30440,31...|[-2567.5049407542...|[1.0,5.2597183293...|       0.0|
|   Happy|A man with a low ...|   406|  0.0|[a, man, with, a,...|[man, low, iq, ac...|(262144,[1076,885...|(262144,[1076,885...|(262145,[1076,885...|[-2910.5645564359...|[1.0,1.6983804901.

In [ ]:
test_results.count()

112

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
test_results.select('label').show()
mood_index.select('mood','label').dropDuplicates().show()

+-----+
|label|
+-----+
|  0.0|
|  0.0|
|  3.0|
|  0.0|
|  1.0|
|  1.0|
|  5.0|
|  0.0|
|  2.0|
|  3.0|
|  6.0|
|  0.0|
|  3.0|
|  5.0|
|  5.0|
|  0.0|
|  1.0|
|  4.0|
|  2.0|
|  0.0|
+-----+
only showing top 20 rows

+----------------+-----+
|            mood|label|
+----------------+-----+
|        Inspired|  1.0|
|        Thrilled|  4.0|
|             Sad|  3.0|
|       Emotional|  6.0|
|        Romantic|  2.0|
|Thrilled/Excited|  7.0|
|           Happy|  0.0|
|           Angry|  5.0|
+----------------+-----+



In [ ]:
test_DF = test_results.select('label').toPandas()
mood_df = mood_index.select('mood','label').dropDuplicates().toPandas()

In [ ]:
mood_df.to_csv('mood_index.csv')
!cp mood_index.csv "drive/My Drive/"

In [ ]:
test_DF.to_csv('test_df.csv')
!cp test_df.csv "drive/My Drive/"